# **NLP1_Run1**

In [ ]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 76.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 6.8 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Ge

In [ ]:

import openai
import os
from IPython.display import Markdown


### **Accessing the GPT4 API**

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))
my_api_key = os.environ['api_key_env']
openai.api_key = my_api_key


### **Generate Mathematical Model**

In [ ]:
problem = """A firm that packs refreshments and beers, situated in the province of Valencia (Spain) employs the same syrup to produce its 1.5 l COLI and PEPSA products on its S1 production line. Once processed, each hectolitre of syrup produces 40 units of the 1.5 l COLI product and 20 units of the 1.5 l PEPSA product. If X1 units of the 1.5 l COLI product and X2 units of the 1.5 l PEPSA product are produced, the firm estimates that the daily income obtained in dollars would be given by the following function:
49000 times X1 minus X1 squared plus 30 times X2 minus two times X2 squared.
It costs 150 dollars to buy and process each hectolitre of syrup. The S1 packaging line has a net capacity of producing 7100 1.5 l product units every hour. The firm works 5 days a week in 8h shifts. Given its weekly target coverage, the firm is committed to produce at least half the amount of PEPSA than COLI. Although priority orders tend to amend its production planning, the firm wishes to have a basic product planning that optimises its daily profits. """

In [ ]:

client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the variables for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 1
)


In [ ]:
print(response1.choices[0].message.content)

1. X1: the number of 1.5 l COLI product units produced.
2. X2: the number of 1.5 l PEPSA product units produced.


In [ ]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the objective function for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 1
)

In [ ]:
print(response2.choices[0].message.content)

The objective function to optimize daily profits is:

Maximize Profit = (49000X1 - X1^2 + 30X2 - 2X2^2) - 150[(X1/40) + (X2/20)]


In [ ]:
response3 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the constraints for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content +  response2.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 1
)

In [ ]:
print(response3.choices[0].message.content)

The constraints for this optimization problem are:

1. The production rate constraint due to the capacity of the S1 production line:
X1 + X2 ≤ 7100 units/hour * 8 hours/day * 5 days/week

2. The raw material availability constraint considering the syrup needed to produce each product:
X1/40 + X2/20 ≤ Available syrup in hectolitres

3. The minimum production requirement of the PEPSA product compared to COLI:
X2 ≥ 0.5X1

4. Non-negativity constraints: 
X1 ≥ 0, X2 ≥ 0

Note: The availability of syrup in hectolitres would need to be specified to formulate the second constraint.


### **Generate Pyomo Code**

In [ ]:
response4 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content + response2.choices[0].message.content + response3.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 1
)

In [ ]:
Markdown(response4.choices[0].message.content)

You can use Pyomo, which is a Python-based open-source optimization modeling language to solve this optimization problem.

Here is a sample Pyomo model for the problem:

```Python
from pyomo.environ import *

# Create a simple model
model = ConcreteModel()

# Define variables 
# Note: You can also set specific lower or upper bounds on the variables at the moment of definition.
model.X1 = Var(domain=NonNegativeReals)
model.X2 = Var(domain=NonNegativeReals)

# Syrup availability is a sample data. Replace 1000 with actual value when available.
Syrup_Availablity = 1000

# Define Objective
model.Profit = Objective(expr = (49000*model.X1 - model.X1**2 + 30*model.X2 - 2*model.X2**2) - 150*((model.X1/40) + (model.X2/20)), sense=maximize)

# Define constraints
model.ProductionRateConstraint = Constraint(expr = model.X1 + model.X2 <= 7100 * 8 * 5)
model.RawMaterialConstraint = Constraint(expr = model.X1/40 + model.X2/20 <= Syrup_Availablity)
model.MinProductionReqConstraint = Constraint(expr = model.X2 >= 0.5*model.X1)

# Solve
SolverFactory('ipopt').solve(model).write()

# Print the values of the decision variables
print("Values of decision variables:")
print("X1 = ", model.X1())
print("X2 = ", model.X2())
```

This model assumes the syrup availability to be 1000 hectolitres. Please replace 1000 with the actual syrup availability when known. Similarly, replace the coefficients of X1 and X2 in the objective function and constraints with actual values if different. The solver 'ipopt' is used in this case, but you can use any other suitable solver as per the requirements.

### **Run the code Generated by GPT4**

In [ ]:
# Install Python API for AMPL
!pip install amplpy --upgrade

from amplpy import AMPL,ampl_notebook

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 28.7 MB/s eta 0:00:00


In [ ]:
_ = load_dotenv(find_dotenv('second_knitro_license.env'))
my_knitro_license = os.environ['knitro_license']

ampl = ampl_notebook(
    modules=["knitro"],  # modules to install
    license_uuid=my_knitro_license,  # license to use
)

Licensed to AMPL Community Edition License for <cevikmelis14@gmail.com>.


In [20]:
from pyomo.environ import *

# Create a simple model
model = ConcreteModel()

# Define variables
# Note: You can also set specific lower or upper bounds on the variables at the moment of definition.
model.X1 = Var(domain=NonNegativeReals)
model.X2 = Var(domain=NonNegativeReals)

# Syrup availability is a sample data. Replace 1000 with actual value when available.
Syrup_Availablity = 1000

# Define Objective
model.Profit = Objective(expr = (49000*model.X1 - model.X1**2 + 30*model.X2 - 2*model.X2**2) - 150*((model.X1/40) + (model.X2/20)), sense=maximize)

# Define constraints
model.ProductionRateConstraint = Constraint(expr = model.X1 + model.X2 <= 7100 * 8 * 5)
model.RawMaterialConstraint = Constraint(expr = model.X1/40 + model.X2/20 <= Syrup_Availablity)

model.MinProductionReqConstraint = Constraint(expr = model.X2 >= 0.5*model.X1)

# Solve
SolverFactory('knitro').solve(model).write()

# Print the values of the decision variables
print("Values of decision variables:")
print("X1 = ", model.X1())
print("X2 = ", model.X2())
print(model.Profit())

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 3
  Number of variables: 2
  Sense: unknown
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Message: Knitro 14.0.0\x3a Locally optimal or satisfactory solution.; objective 400289176.03887665; feasibility error 0; 9 iterations; 0 function evaluations
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 0.02893853187561035
# ----------------------------------------------------------
#   Solution Information
# -----------------------------

### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**